In [1]:
from collections import defaultdict

## see examples in https://www.gurobi.com/resource/modeling-examples-using-the-gurobi-python-api-in-jupyter-notebook/
import gurobipy as gp
from gurobipy import GRB

import math
import sys

from fruit_distribution import *   # import module to create the various desired fruit distributions 

# tested with Python 3.7.0 & Gurobi 9.0

## based on the Gurobi technician routing scheduling example
# https://gurobi.github.io/modeling-examples/technician_routing_scheduling/technician_routing_scheduling.html

In [2]:
## Constants 
Td      = 2.       # fruit handling time
M       = 600     # arbitrarily large number
v_vy    = 0.3     # in m/s, vehicle velocity along orchard row

v_max   = 0.5
a_max   = 1.
t_grab  = 0.1 

n_row   = 1       # total number of horizontal rows with cells containg one arm per cell
n_arm   = 1       # number of arms in one horizontal row

x_lim = [0.2, 1.2]
y_lim = [0. , 12.]
z_lim = [0., 2.]

density = 3       # in fruit/m^2

l       = 0.3     # in m, length of the cell along the orchard row (y-axis), parallel to vehicle travel
w       = 2.      # in m, width/height of the horizontal row of arms (z-axis) perpendicular to vehicle travel

cell_h    = l
cell_w    = w
arm_reach = 1.

## set fruit distribution flag
# 0     == Raj's digitized fruits
# 1     == uniform random  (if algorithm == 1, use melon version)
# 2     == uniform random, equal cell density
# 3     == multiple densities separated by some space (only melon for now)
# 4     == fruit in vertical columns
# 5     == "melon" version of columns (user inputs desired no. fruits, z height, and distance between fruit in y-coord)
set_distribution = 5

## set algorithm being used (can add more later): ##
# 1     == melon
# not 1 == not melon
set_algorithm    = 1


In [3]:
## Functions
def getRNGSeedList(n_runs):
        '''
           Open the random seed list rngseed_list_20200901.csv with 200 seeds for each of the 3 real fruit coordinate axis
           and 3 fake fruit coordinate axis.
        '''
        # keeps track of the row number of the csv being read (each row contains the seeds for one run)
        csv_i     = 0

        seed_list = list()

        with open('./rngseed_list_20200901.csv') as csvfile:
            reader = csv.reader(csvfile, delimiter=',', quoting=csv.QUOTE_NONNUMERIC)
            for row in reader:
                seed_list.append(row)
                if csv_i == n_runs:
                    break

                csv_i += 1

        # print(seed_list)
        return(seed_list)
    
    
def createFruit(fruitD, set_algorithm, set_distribution, density, x_seed, y_seed, z_seed):
        if set_distribution == 0:
            csv_file = './TREE_FRUIT_DATA/apple_tree_data_2015/Applestotheright.csv'
            [numFruit, sortedFruit] = fruitD.csvFile(csv_file, 0)

        elif set_distribution == 1:
            if set_algorithm == 1:
                [numFruit, sortedFruit] = fruitD.uniformRandomMelon(density, y_seed, z_seed)
            else:
                [numFruit, sortedFruit] = fruitD.uniformRandom(density, x_seed, y_seed, z_seed)
            # print()
            # print('--------------------------------------------')
            # print('Number of fruit:', numFruit)
            # print()

        elif set_distribution == 2: 
            fruit_in_cell = math.ceil(density * (cell_h*cell_l*arm_reach)) # num of fruit in front of cell if using (equalCellDensity())
            print('Number of fruit in each cell:', fruit_in_cell)
            print()
            [numFruit, sortedFruit] = fruitD.equalCellDensity(n_row, n_arm, cell_h, cell_l, arm_reach, fruit_in_cell, x_seed, y_seed, z_seed)

        elif set_distribution == 3: 
            densities = np.array([5, 4, 3])
            [numFruit, sortedFruit] = fruitD.uniformRandomMelon_MultipleDensity(densities, y_seed, z_seed)

        elif set_distribution == 4: 
            [numFruit, sortedFruit] = fruitD.column(v_vy, v_max, a_max, t_grab, n_row, n_arm, cell_h, z_seed)
            
        elif set_distribution == 5:
            n_fruit = 4
            d_y  = Td*v_vy*(n_fruit+1)/(n_fruit+2) # kind of works 3/4 or 5/8 fruit with 1 arm: (Td/2)*v_vy
            print('with Td', Td, 'and v_vy', v_vy)
            print('d_y for this line of fruit:', d_y, 'so the total distance they take up:', d_y*n_fruit)
            z_coord = cell_w / 2
            [numFruit, sortedFruit] = fruitD.column_melon(n_fruit, d_y, z_coord)

        else: 
            print('not a correct fruit distribution, defaulting to uniform random')
            if set_algorithm == 1:
                [numFruit, sortedFruit] = fruitD.uniformRandomMelon(density, y_seed, z_seed)
            else:
                [numFruit, sortedFruit] = fruitD.uniformRandom(density, x_seed, y_seed, z_seed)

        return([numFruit, sortedFruit])
    
    
def calc_TW(arm_n, y_coord, v_vy):
    TW_start = (y_coord + (arm_n - 1)*l) / v_vy
    TW_end   = (y_coord + arm_n*l) / v_vy
    return([TW_start, TW_end])


def calc_tki(arm_n, y_coord, tkj):
    tki = math.max(tkj + Td , (y_coord + (arm_n - 1)*l) / v_vy)
    return(tki)

In [4]:
class Arm():
    def __init__(self, row_n, arm_n):
        self.row_n = row_n
        self.arm_n = arm_n

    def __str__(self):
        return f"Arm: {self.arm_n}\n Horizontal Row Number: {self.row_n}"

In [5]:
class Fruit():
    def __init__(self, index, y_coord, z_coord):#, job, tStart, tEnd, tDue):
        self.index = index       # fruit's index when ordered by y-coordinate
        self.y_coord = y_coord   # y-coordinate of the fruit
        self.z_coord = z_coord
        
    def __str__(self):
        return f"Fruit Index: {self.index}\n  Y-axis location: {self.y_coord}\n"

In [6]:
class Job():
    def __init__(self, fruit_i, arm_k, v_vy, l):
        self.fruit_i = fruit_i
        self.arm_k   = arm_k
        # k+1 was added because the MIP model in paper assumes k starts at 1
        self.TW_start = (self.fruit_i.y_coord + (self.arm_k.arm_n - 1 + 1)*l) / v_vy
        self.TW_end   = (self.fruit_i.y_coord + (self.arm_k.arm_n + 1)*l) / v_vy

In [7]:
## Get fruit list based on desired distribution
n_runs = 1

seed_list = getRNGSeedList(n_runs)

for run in range(n_runs):
    # get seeds for x, y, and z RNG (probably unneccessary right now, especially for x)
    seed = [seed_list[run][0], seed_list[run][1], seed_list[run][2]]
    x_seed = PCG64(int(seed[0]))
    y_seed = PCG64(int(seed[1]))
    z_seed = PCG64(int(seed[2]))

fruitD = fruitDistribution(x_lim, y_lim, z_lim)
[numFruit, sortedFruit] = createFruit(fruitD, set_algorithm, set_distribution, density, x_seed, y_seed, z_seed)

print('Total fruit in the orchard row',numFruit)
print()
print('List of the x, y, and z coordinates of the sorted fruit')
print(sortedFruit)

with Td 2.0 and v_vy 0.3
d_y for this line of fruit: 0.5 so the total distance they take up: 2.0
fruit y-coordinates: [0.  0.5 1.  1.5]
Total fruit in the orchard row 4

List of the x, y, and z coordinates of the sorted fruit
[[1.  1.  1.  1. ]
 [0.  0.5 1.  1.5]
 [1.  1.  1.  1. ]
 [0.  1.  2.  3. ]
 [0.  0.  0.  0. ]]


In [8]:
## create arm object list
arm = list()

for r in range(n_row):
# for r in range(1, n_row+1):
    for k in range(n_arm):
#     for k in range(1, n_arm+1):
        this_arm = Arm(r, k)
        arm.append(this_arm)
    
# print(arm)

In [9]:
## create fruit object list
fruit = list()

for index in range(numFruit):
    y_coord = sortedFruit[1][index]
    z_coord = sortedFruit[2][index]
    this_fruit = Fruit(index, y_coord, z_coord)
#     print('Fruit index', index, 'should match this index', sortedFruit[3][index])
#     print('with y and z coordinates:', y_coord, z_coord)

    fruit.append(this_fruit)

# print(fruit)

In [10]:
## create job oject list
job = list()

for k in arm:
    for i in fruit:       
        this_job = Job(i, k, v_vy, l)
#         print('for arm', this_job.arm_k.arm_n, 'and fruit', this_job.fruit_i.index)
#         print('TW starts at', this_job.TW_start, 'and TW ends at', this_job.TW_end)
        job.append(this_job)
    
# print(len(job))
# print(len(arm)*len(fruit))

In [11]:
def solve_melon_mip(arm, fruit, job):
    ## Build useful data structures
    # lists:
    K = [k.arm_n for k in arm]      # list of arm numbers
    N = [i.index for i in fruit]    # list of fruit indexes
    Y = [i.y_coord for i in fruit]  # list of fruits' y-coordinate (x-coordinate in the paper)
#     print('number of arms:',K, 'with length', len(K))
#     print()
#     print('number of fruits:',N, 'with length', len(N))
#     print()
#     print('fruit y-coordinate:', Y, 'with length', len(Y))
#     print()
    
    # dictionaries:
    # TW start and end are dependant on both the arm number and fruit number, requiring dictionary with a list
    # see canCover in technician gurobi example[k]
    TW_start = {i : [j.TW_start for j in job if j.fruit_i.index == i] for i in N}
    TW_end   = {i : [j.TW_end for j in job if j.fruit_i.index == i] for i in N}
    print('TW start', TW_start, 'with length', len(TW_start))
    print()
    print('TW end', TW_end, 'with length', len(TW_end))
    print()
    print('so the third fruit, picked by the first arm TW_start should be:', TW_start[3][0])
    print()

    
    ### Create model
    m = gp.Model("melon_mip")
    
    
    ### Decision variables
    # Arm-fruit assignment (is fruit i picked by arm k)
    x = m.addVars(K, N, vtype=GRB.BINARY, name="x")
    
    # Time arm k reaches fruit i
    t = m.addVars(K, N, lb=0, name="t")
    
    
    ### Constraints
    # At most one arm can be assigned to a fruit (1)
    m.addConstrs((x.sum('*', i) <= 1 for i in N), name="assignOne")
    
    # Time elapsed between pickup of any two fruit reached by the same arm is at least Td (2)
    m.addConstrs((t[k, i] + Td - t[k, j] <= M * (2 - x[k, j] - x[k, i]) for i in N for j in N for k in K if Y[j] > Y[i]), name="atLeast")

    # Ensure each node is visited within the given time window (3) and (4)
    # TW_start and TW_end are matching the fruit index number exactly (disctionary), so [2][0] == index 2 (even 
    # though it starts at zero, second arm back from 0th arm)  
    m.addConstrs((t[k, i] <= max(TW_start[i][k], TW_end[i][k]) for i in N for k in K), name="timeWinA")
    m.addConstrs((t[k, i] >= min(TW_start[i][k], TW_end[i][k]) for i in N for k in K), name="timeWinB")

    
    ### Objective function
    m.setObjective(gp.quicksum(x[k, i] for i in N for k in K) , GRB.MAXIMIZE)
    
    ## write model into a file
    # see https://www.gurobi.com/documentation/9.5/refman/py_model_write.html
    # https://www.gurobi.com/documentation/9.5/refman/model_file_formats.html
    m.write("melon_mip.lp")
    m.write("melon_mip.mps")
    m.optimize()

    status = m.Status
    if status in [GRB.INF_OR_UNBD, GRB.INFEASIBLE, GRB.UNBOUNDED]:
        print("Model is either infeasible or unbounded.")
        sys.exit(0)
    elif status != GRB.OPTIMAL:
        print("Optimization terminated with status {}".format(status))
        sys.exit(0)
        
        
    ### Print results
    # Assignments
    picked = 0
    
    print()
    for j in job:
        if x[j.arm_k.arm_n, j.fruit_i.index].X > 0:
            print('fruit', j.fruit_i.index, 'assigned to arm', j.arm_k.arm_n, 'at t = ', t[j.arm_k.arm_n, j.fruit_i.index].X)
            picked += 1
            
    print()            
    print('a total of', picked, 'fruit were harvested out of', len(N))
    print()
    print('model variables:', m.getAttr("x", m.getVars()))
    print()
#     print('with a length:', (len(m.getAttr("x", m.getVars()))/2)/len(K))
    
    # check that TW and t^k_i match indexes and arms
#     print()
#     for k in K:
#         for i in N:
#             print('TW start:', TW_start[i][k], 'TW end:', TW_end[i][k], 'and t^k_i', t[k, i].X)


In [12]:
def solve_melon_mip_read():
    #### use solution file from solve_melon_mip to see what's going on
    ### Create model
    m = gp.read('wall_mip.mps')
    m.read('wall_start.mst')
    
    m.optimize()

    status = m.Status
    if status in [GRB.INF_OR_UNBD, GRB.INFEASIBLE, GRB.UNBOUNDED]:
        print("Model is either infeasible or unbounded.")
        sys.exit(0)
    elif status != GRB.OPTIMAL:
        print("Optimization terminated with status {}".format(status))
        sys.exit(0)
        
    print()
        
        
    ### Print results
    # Assignments
    picked = 0
    
    print('model variables:', m.getAttr("x", m.getVars()))
    print()


In [13]:
def printScen(scenStr):
    sLen = len(scenStr)
    print("\n" + "*"*sLen + "\n" + scenStr + "\n" + "*"*sLen + "\n")

In [14]:
if __name__ == "__main__":
    # Base model
    printScen("Solving base scenario model")
    solve_melon_mip(arm, fruit, job)
#     solve_melon_mip_read()


***************************
Solving base scenario model
***************************

TW start {0: [0.0], 1: [1.6666666666666667], 2: [3.3333333333333335], 3: [5.0]} with length 4

TW end {0: [1.0], 1: [2.666666666666667], 2: [4.333333333333334], 3: [6.0]} with length 4

so the third fruit, picked by the first arm TW_start should be: 5.0

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2022-02-07
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 18 rows, 8 columns and 36 nonzeros
Model fingerprint: 0x344e88c1
Variable types: 4 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 18 rows and 8 columns
Presol

## 